# Initialization

In [1]:
import vertexai
from vertexai.preview import reasoning_engines

PROJECT_ID = "hello-world-418507"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://2025-wy-agentspace"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [2]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (41 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash-exp",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "I can answer your questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

In [3]:

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

## Local test

In [ ]:
session = app.create_session(user_id="u_123")
session

In [ ]:
app.list_sessions(user_id="u_123")


In [ ]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

In [ ]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in london",
):
    print("*"*100)
    print(event)

In [ ]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="whats the weather in new york",
):
    print("*"*100)
    print(event)

## Deploy

- reason engine id: 3326620808347910144

In [ ]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"   
    ]
)

## Test in new session

In [ ]:
live_app = vertexai.agent_engines.get('projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144')
session = live_app.create_session(user_id="u_123")
session

In [ ]:
for event in live_app.stream_query(
    user_id="u_123",
    session_id=session["id"],
    message="whats the weather in london",
):
    print("*"*100)
    print(event)

## Logging

```bash
resource.type="aiplatform.googleapis.com/ReasoningEngine"
resource.labels.location="us-central1"
resource.labels.reasoning_engine_id="2590282269272834048"
```

In [58]:
from google.cloud import logging
from datetime import datetime, timedelta
import pytz

def get_recent_logs(engine_id, minutes=5):
    """
    Retrieve logs from a specific Reasoning Engine for the last specified minutes.
    
    Args:
        engine_id (str): The ID of the Reasoning Engine
        minutes (int): Number of minutes to look back for logs
    
    Returns:
        List of log entries
    """
    # Initialize the logging client
    client = logging.Client()
    
    # Calculate the time range in Pacific Time
    vegas_tz = pytz.timezone('America/Los_Angeles')
    end_time = datetime.now(vegas_tz)
    start_time = end_time - timedelta(minutes=minutes)
    
    # Format the start timestamp for the query
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    # Construct the filter string
    filter_str = (
        'resource.type="aiplatform.googleapis.com/ReasoningEngine" '
        'resource.labels.location="us-central1" '
        f'resource.labels.reasoning_engine_id="{engine_id}" '
        f'timestamp >= "{start_time_str}"'
    )
    
    # Create an iterator for the logs and convert to list
    entries = list(client.list_entries(filter_=filter_str))
    
    print(f"Found {len(entries)} log entries in the last {minutes} minutes")
    
    # Print just the payload for each log entry
    for i, entry in enumerate(entries, 1):
        print(f"\n--- Log Entry {i} ---")
        print(f"Payload: {entry.payload}")



In [ ]:
# Get logs for the last 15 minutes
get_recent_logs(engine_id="3326620808347910144", minutes=5)

## Agentspace Registration

In [ ]:
%%bash
export PROJECT_ID="hello-world-418507" 
export PROJECT_NUMBER="671247654914" 
export REASONING_ENGINE="projects/671247654914/locations/us-central1/reasoningEngines/3326620808347910144" 
export AGENT_DISPLAY_NAME="Wei WeatherAgent"
export AGENT_DESCRIPTION="Agent to answer questions about the time and weather in a city."
export AGENT_ID="wei_weather_agent" 
export AS_APP="2025-abc-gds_1741922567491" 

curl -X PATCH -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "x-goog-user-project: ${PROJECT_ID}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant?updateMask=agent_configs -d '{
    "name": "projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant",
    "displayName": "Default Assistant",
    "agentConfigs": [{
      "displayName": "'"${AGENT_DISPLAY_NAME}"'",
      "vertexAiSdkAgentConnectionInfo": {
        "reasoningEngine": "'"${REASONING_ENGINE}"'"
      },
      "toolDescription": "'"${AGENT_DESCRIPTION}"'",
      "icon": {
        "uri": "https://fonts.gstatic.com/s/i/short-term/release/googlesymbols/corporate_fare/default/24px.svg"
      },
      "id": "'"${AGENT_ID}"'"
    }]
  }'

## Delete Agent Engine

In [ ]:
remote_app.delete(force=True)